In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def mostrar(img, cmap='gray', titulo=''):
    plt.imshow(img, cmap=cmap)
    plt.title(titulo)
    plt.axis('off')
    plt.show()

# 1. Cargar imagen
img = cv2.imread('sudoku.jpg')
img_original = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 2. Preprocesar: desenfoque + umbral adaptativo
blur = cv2.GaussianBlur(gray, (7, 7), 3)
thresh = cv2.adaptiveThreshold(blur, 255, 
                                cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                cv2.THRESH_BINARY_INV, 11, 2)

# 3. Encontrar contornos
contornos, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 4. Filtrar el contorno más grande (debería ser el tablero)
contorno_tablero = max(contornos, key=cv2.contourArea)

# 5. Aproximar el contorno a un polígono (idealmente un cuadrado)
peri = cv2.arcLength(contorno_tablero, True)
approx = cv2.approxPolyDP(contorno_tablero, 0.02 * peri, True)

if len(approx) == 4:
    pts = approx.reshape(4, 2)
    
    # Ordenar puntos: superior izq, superior der, inf der, inf izq
    def ordenar_puntos(pts):
        suma = pts.sum(axis=1)
        resta = np.diff(pts, axis=1)

        ordenados = np.zeros((4, 2), dtype="float32")
        ordenados[0] = pts[np.argmin(suma)]
        ordenados[2] = pts[np.argmax(suma)]
        ordenados[1] = pts[np.argmin(resta)]
        ordenados[3] = pts[np.argmax(resta)]
        return ordenados

    pts_ordenados = ordenar_puntos(pts)

    # 6. Transformación de perspectiva
    lado = 450
    destino = np.array([[0, 0], [lado-1, 0], [lado-1, lado-1], [0, lado-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(pts_ordenados, destino)
    warp = cv2.warpPerspective(img_original, M, (lado, lado))

    mostrar(warp, titulo='Tablero de Sudoku enderezado')
else:
    print("No se detectó un tablero cuadrado.")


No se detectó un tablero cuadrado.
